<a href="https://colab.research.google.com/github/vnaticzhock/quantitative-trading/blob/main/1216_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsmodels --upgrade
!pip install pmdarima
!pip install arch

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
## Augmented Dickey-Fuller Test
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model
import statsmodels.graphics.tsaplots as sgt
import pmdarima as pmd
import matplotlib.pyplot as plt

In [ ]:
def timeFormat(time: str) -> str:
  tmp = time.split('/')
  tmp[0] = str(int(tmp[0]) + 1911)
  return '-'.join(tmp)

df = pd.read_csv('/content/日收盤價及成交量等.csv')
df['Unnamed: 0'] = df['Unnamed: 0'].apply(lambda x: timeFormat(x))
df.index = pd.Index(df['Unnamed: 0'].copy())
df = df.sort_index()
dta = pd.DataFrame(df['Close-Price'].pct_change().dropna())
dta.columns = ['日報酬率']
dta.index.name = '年月日'

dta.tail(3)

In [ ]:
price_p_value = adfuller(dta['日報酬率'])[1]
if price_p_value > 0.05:
    print('Nonstationary')
else:
    print('Stationary')

In [ ]:
pmd_mdl = pmd.auto_arima(dta['Close-Price'], stationary=True)

In [ ]:
print(pmd_mdl.summary())

In [ ]:
model = ARIMA(dta['Close-Price'], order=(1, 0, 1))
stats_mdl = model.fit()
print(stats_mdl.summary())

stats_mdl.plot_diagnostics(figsize = (15, 10))
plt.show()

In [ ]:
pmd_residual = pmd_mdl.arima_res_.resid
white_noise = acorr_ljungbox(pmd_residual, lags = [10], return_df=True)
print(white_noise)

In [ ]:
LM_pvalue = het_arch(pmd_residual, ddof = 4)[1]
print('LM-test-Pvalue:', '{:.5f}'.format(LM_pvalue))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (18,5))

sgt.plot_acf(pmd_residual**2, zero = False, lags = 40, ax=ax[0])
sgt.plot_pacf(pmd_residual**2, zero = False, lags = 40, ax=ax[1])
plt.show()

In [ ]:
mdl_garch = arch_model(pmd_residual, vol='GARCH', p=1, q=1)
garch_fit = mdl_garch.fit()
print(garch_fit.summary())

In [ ]:
import statsmodels.api as sm

garch_std_resid = pd.Series(garch_fit.resid / garch_fit.conditional_volatility)
fig = plt.figure(figsize = (15, 8))

# Residual
garch_std_resid.plot(ax = fig.add_subplot(3,1,1), title = 'GARCH Standardized-Residual', legend = False)

# ACF/PACF
sgt.plot_acf(garch_std_resid, zero = False, lags = 40, ax=fig.add_subplot(3,2,3))
sgt.plot_pacf(garch_std_resid, zero = False, lags = 40, ax=fig.add_subplot(3,2,4))

# QQ-Plot & Norm-Dist
sm.qqplot(garch_std_resid, line='s', ax=fig.add_subplot(3,2,5)) 
plt.title("QQ Plot")
fig.add_subplot(3,2,6).hist(garch_std_resid, bins = 40)
plt.title("Histogram")

plt.tight_layout()
plt.show()

In [ ]:
white_noise_garch = acorr_ljungbox(garch_std_resid, lags = [10], return_df=True)
white_noise_garch